# ARGENTINE GOVERNMENT DATA (OIL AND GAS)

CSV data set used in this notebook downloable at: https://bit.ly/3UwPbbG 

Data property of:
*Secretaría de Energía. Subsecretaría de Planeamiento Energético. Dirección Nacional de Escenarios y Evaluación de Proyectos. Dirección de Información Energética.Tecnología de la Información.*

In [1]:
from IPython.display import Image
#Esta es la imagen satelital y el área de consesion (intentar con QGIS)
#Image('img/DSC_0330.JPG', width=1200 , height=800)

# EDA

This is a very complete and large data set, so our first step consists of exploratory data analysis (EDA). Although data is provided at a monthly frequency, to reduce complexity we will work with data on a yearly frequency.

In [22]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import geopandas as gpd
import urllib
import missingno as msno
from termcolor import colored

# Set stylistic options for plots generated later on
sns.set_context("paper")
plt.style.use('fivethirtyeight')

In [36]:
# Check for data
data_url = 'http://datos.energia.gob.ar/dataset/c846e79c-026c-4040-897f-1ad3543b407c/resource/b5b58cdc-9e07-41f9-b392-fb9ec68b0725/download/produccin-de-pozos-de-gas-y-petrleo-no-convencional.csv'
datafile = False


# Define some colors for later on
class color:
    GREEN = 'green'
    RED = 'red'
    BOLD = 'bold'


def check_and_load_dataset():
    try:
        df = pd.read_csv('arg_gov_oil_and_gas_dataset.csv')
        print(colored('\n===== DATA SET FOUND! =====', color.GREEN))
        print('Reading into data frame...')
        print(colored('Data read successfully!', color.GREEN))
        return df, True
    except FileNotFoundError:
        print(colored('\n====== DATA SET NOT FOUND OR UNREADABLE! ======', color.RED))
        return None, False

def download_dataset(data_url):
    print('Downloading data set. This may take a while, please wait...')
    try:
        urllib.request.urlretrieve(data_url, 'arg_gov_oil_and_gas_dataset.csv')
        print(colored('Data set downloaded successfully!', color.GREEN))
        return pd.read_csv('arg_gov_oil_and_gas_dataset.csv'), True
    except urllib.error.URLError:
        print(color('\n====== COULD NOT DOWNLOAD DATA SET! Check internet connection and/or file/directory permissions and try again. =====', Color.RED + Color.BOLD))
        return None, False

def user_prompt():
    while True:
        data_prompt = input('\nDownload the dataset (approximately 100MB)? [yes]/no: ').lower()
        if data_prompt in ['yes', 'y', '']:
            return True
        elif data_prompt == 'no':
            print(colored('\n====== DATA SET NOT DOWNLOADED! Please download the dataset manually and place it in the working directory before proceeding. =====', Color.RED + Color.BOLD))
            return False
        else:
            print(colored('Invalid input. Try again.', color.RED))

def main():
    global data_url  # Declare data_url as a global variable
    df, datafile = check_and_load_dataset()

    if not datafile:
        if user_prompt():
            df, datafile = download_dataset(data_url)

    # Rest of your main code here
    if datafile:
        # Continue with the rest of your analysis or processing

        if __name__ == "__main__":
            data_url = 'http://datos.energia.gob.ar/dataset/c846e79c-026c-4040-897f-1ad3543b407c/resource/b5b58cdc-9e07-41f9-b392-fb9ec68b0725/download/produccin-de-pozos-de-gas-y-petrleo-no-convencional.csv'

In [37]:
main()


====== DATA SET NOT FOUND OR UNREADABLE! ======

Download the dataset (approximately 100MB)? [yes]/no: yes
Data set downloaded successfully!


In [ ]:
df.head(5)

In [ ]:
msno.bar(df, color = 'g')

In [7]:
for column in df.columns:
    num_distinct_values = len(df[column].unique())
    print(f"{column}: {num_distinct_values} distinct values")

idempresa: 41 distinct values
anio: 18 distinct values
mes: 12 distinct values
idpozo: 3807 distinct values
prod_pet: 89833 distinct values
prod_gas: 136875 distinct values
prod_agua: 84554 distinct values
iny_agua: 226 distinct values
iny_gas: 148 distinct values
iny_co2: 1 distinct values
iny_otro: 1 distinct values
tef: 13891 distinct values
vida_util: 467 distinct values
tipoextraccion: 12 distinct values
tipoestado: 17 distinct values
tipopozo: 7 distinct values
observaciones: 197 distinct values
fechaingreso: 8519 distinct values
rectificado: 2 distinct values
habilitado: 1 distinct values
idusuario: 41 distinct values
empresa: 41 distinct values
sigla: 3623 distinct values
formprod: 26 distinct values
profundidad: 2285 distinct values
formacion: 26 distinct values
idareapermisoconcesion: 107 distinct values
areapermisoconcesion: 107 distinct values
idareayacimiento: 146 distinct values
areayacimiento: 145 distinct values
cuenca: 4 distinct values
provincia: 6 distinct values
coo

In [ ]:
# Check what the columns are and what shape our dataset is in
print('Columns in the data frame.')
print(df.columns,df.shape)


In [ ]:
# We don't need all these columns. Let's take what we need for now, and leave what we don't behind (commented out)
df=df[['idempresa', 'anio', 'mes', 'idpozo', 'prod_pet', 'prod_gas',
       'prod_agua',
      #'iny_agua', 'iny_gas', 'iny_co2', 'iny_otro', 'tef',
       #'vida_util', 'tipoextraccion', 
      'tipoestado', 'tipopozo',
      # 'observaciones', 'fechaingreso', 'rectificado', 'habilitado',
      # 'idusuario',
      'empresa', 'sigla', 'formprod', 'profundidad', 'formacion',
       'idareapermisoconcesion', 'areapermisoconcesion', 
      'idareayacimiento', 'areayacimiento', 'cuenca', 'provincia', 'coordenadax', 'coordenaday',
       'tipo_de_recurso', 'proyecto', 'clasificacion', 'subclasificacion',
       'sub_tipo_recurso', 'fecha_data']].copy()

In [ ]:
# Drop an index if it doesn't have a valid anio/año (year) or mes (month) columns since that makes it are useless to us.
df = df.dropna(subset=['anio', 'mes'])

In [ ]:
# Our data set has some problems with the anio column, let's make sure it's a valid int
df['anio'] = pd.to_numeric(df['anio'], errors='coerce').astype(int)
# Generate fecha (data) datetime-typed column out of anio and mes
df['fecha'] = pd.to_datetime(df['anio'].astype(str) + '-' + df['mes'].astype(str))
# Now let's see...
print('Dates we generated, sorted from oldest to newest. They should be formated YYYY-MM-DD if all is right.\n')
print(df.sort_values(by=['fecha'])['fecha'])

In [ ]:
# Let's check total invalid entries for each column
print('Sum of invalid entries per column. Looks like nothing that is of primary interest to us has invalid entries.\n')
print(df.isna().sum())

In [ ]:
def null_count():
    return pd.DataFrame({'features': df.columns,
                'dtypes': df.dtypes.values,
                'NaN count': df.isnull().sum().values,
                'NaN percentage': df.isnull().sum().values/df.shape[0]}).style.background_gradient(cmap='Set3',low=0.1,high=0.01)
null_count()

In [ ]:
df.describe().T.style.background_gradient(axis=0, cmap='Set2')

In [ ]:
# Generate axes ticks
tick_spacing = 4
ticks = [df['anio'].min() - 1]
while np.max(ticks) < df['anio'].max():
    ticks.append(np.max(ticks) + tick_spacing)

# Generate the plot
fig, ax = plt.subplots()
sns.boxplot( x = df['anio'], 
            color='#5092AB',
            #linecolor = 'black',
            flierprops={"marker": "D", "markerfacecolor" : "#5092AB"}
           )
ax.set_xlim(np.min(ticks) - 1, np.max(ticks) + 1)    # Need to +-1 to make sure ends of graph and ticks not cut off
plt.xticks(ticks)
plt.title('Range of years present in dataset')
plt.xlabel('Year')
plt.show()

In [ ]:
# Filter out data with outlier years via inter-quartile range (IRQ)

#Calculate IQR
Q1 = df['anio'].quantile(0.25)
Q3 = df['anio'].quantile(0.75)
IQR = Q3 - Q1

# Define upper and lower limits for identifying outliers
lower_lim = Q1 - 1.5 * IQR
upper_lim = Q3 + 1.5 * IQR

# Filter the outliers from the dataframe
df['anio']=df['anio'][(df['anio'] >= lower_lim)]

hist_df = df.groupby(['anio']).size().reset_index(name='count')
hist_df['anio'] = (hist_df['anio'].astype(int)).astype(str)

#print(hist_df)


# Plot a histogram
fig = sns.barplot(data = hist_df,
                  x = 'anio',
                  y = 'count',
                  #hue = 'count',
                  width = 1,
                  palette = 'coolwarm',
                  alpha = 0.8
                 )


sns.histplot(data=hist_df, x='anio', bins=10, kde=False, color='blue', alpha=0.8)
plt.xlabel('Year')
plt.ylabel('# of data points')
fig.set_xticklabels(hist_df['anio'], rotation=60)
plt.title('Distribution of data points according to year')
plt.show()

In [ ]:
df.to_csv('df_EDA.csv',header=True)

# EXTRA INFORMATION ABOUT DATASET

## We can see that, thankfully, most data is recent.

## We're gonna keep from the entries from the three companies with the most data.

In [ ]:
# Group and count number of datapoints by company ID, then sort by count
df_idempresa = df.groupby(['idempresa']).size().reset_index(name='datapoints').sort_values(by = ['datapoints'], ascending = False)

# List the top three companies
print('Top three companies in terms of datapoints')
print(df_idempresa.head(3))

## The three companies with the most datapoints are YPF, APS, and PLU. We'll proceed accordingly.

In [ ]:
YPF = df.query('idempresa=="YPF" and 2010 <= anio')
APS = df.query('idempresa=="APS" and 2010 <= anio')
PLU = df.query('idempresa=="PLU" and 2010 <= anio')

# Create a histogram 
fig = sns.barplot(data = df_idempresa,
                  x = "idempresa", 
                  y = "datapoints",
                  #hue = 'datapoints',
                 # hue_norm = LogNorm(vmin=df_idempresa['datapoints'].min(), vmax=df_idempresa['datapoints'].max()),
                  palette = 'coolwarm',
                  alpha = 0.8,
                  #legend = None,
                  width = 1)
fig.set_yscale("log")
plt.grid(which = 'both', axis = 'y')
plt.title('Datapoints by company')
plt.xlabel('Company ID')
plt.ylabel('# of datapoints')
plt.xticks(rotation=90, fontsize=8)
plt.tight_layout()
plt.show()

## Note the above graph is *logarithmic* in scale. 

## Let's concatenate data from the top three companies into a new data frame.


In [ ]:
companies = ['YPF','APS','PLU']
df_final=df.query('idempresa == @companies and 2010 <= anio <= 2023').sort_values(by = ['idpozo']).reset_index()

#Let's see the "final" data frame after all our filtering
df_final

## We graph production by company by year. Again, note the logarithmic scale of the plot.

In [ ]:
df_prod = df_final.groupby(['idempresa', 'fecha'])['prod_pet'].sum().reset_index()
companies = df_prod['idempresa'].unique()
years = np.sort(df_final['anio'].unique().astype(int))

plt.subplots(figsize = (10,10))
fig = sns.histplot(data = df_prod,
                   stat = 'count',
                   x = 'fecha',
                   multiple = 'layer',
                   hue = 'idempresa',
                   weights = 'prod_pet',
                   palette = 'coolwarm_r',
                   alpha = 0.7,
                   bins = len(years),
                   legend = False
                   )
plt.legend(title='Company ID', loc='upper left', labels=['YPS', 'PLU', 'APS'])     # Can double check correct via legend = True inside plot and commenting this line out
fig.set_yscale("log")
plt.grid(which = 'both', axis = 'both')
plt.title('Petroleum produced by top 3 copmpanies per year')
plt.xlabel('Year')
plt.ylabel('Petroleum production in $m^3$')

# We are interested in observing oil production about depth to determine at what depth the productive formations are located and to assess the correlation between production and depth

In [ ]:
df_depth = df_final.groupby(['profundidad'])['prod_pet'].sum().reset_index()

# Tu código original con sns.lineplot modificado a sns.scatterplot
fig = sns.scatterplot(data=df_depth,
                      x='profundidad',
                      y='prod_pet',
                      palette='coolwarm_r',
                      alpha=0.7,
                      legend=False
                      )

plt.grid(True, which='both', axis='both')
plt.show()

# Suponiendo que df_depth es tu DataFrame
correlation_matrix = df_depth[['profundidad', 'prod_pet']].corr()

# Mostrar la matriz de correlación
print(correlation_matrix)
# Obtener el coeficiente de correlación específico entre 'profundidad' y 'prod_pet'
correlation_value = correlation_matrix.loc['profundidad', 'prod_pet']
print(f"Coeficiente de correlación entre 'profundidad' y 'prod_pet': {correlation_value}")


#LA CORRELACION DE LA PRODUCCION DE PETROLEO CON LA PROFUNDIAD ES MODERADA A ALTA